### Train a ALM in Google Colab!

### Clone the repository if you don't have it already

In [1]:
import os

if not os.path.isdir('nanoALM'):
    !git clone https://github.com/LWL220184016/nanoVLM_From_Huggingface.git
%cd nanoVLM_From_Huggingface/
!ls

fatal: destination path 'nanoVLM_From_Huggingface' already exists and is not an empty directory.
/content/nanoVLM_From_Huggingface
assets			checkpoints  generate.py      nanoALM.ipynb  test
benchmark-inference.py	data	     measure_vram.py  old
benchmark_suite.py	debug	     models	      README.md


### Imports and Setup

In [2]:
# Let's authentificate with the Hugging Face Hub so you can push your model
# from huggingface_hub import notebook_login
# notebook_login()
# !huggingface-cli login


In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')

check_dir = '/content/stage1'
source_dir = '/content/drive/MyDrive/nanoALM/7/stage1'

# 檢查來源資料夾是否存在且非空
if os.path.isdir(check_dir) and os.listdir(check_dir):
    print(f"偵測到檔案存在於 '{check_dir}'。將停止複製檔案和後續 pip 安裝。")
else:
    print(f"'{check_dir}' 是空的或不存在。將複製檔案並繼續執行 pip 安裝。")
    !cp -r {source_dir} /content

    # If you get an "Error" from pip's dependency resolver but the cell complets fine, this is not an issue, you can continue :)
    !pip -q install torch
    !pip -q install gcsfs
    !pip -q install tqdm
    !pip -q install huggingface_hub
    !pip -q install librosa
    !pip install soundfile librosa -q
    # !pip install --upgrade transformers
    !pip install datasets==3.6.0

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
偵測到檔案存在於 '/content/stage1'。將停止複製檔案和後續 pip 安裝。


In [4]:
# Decide on the name of your model here!
# You will need your HF user name and the name you want to give to it
# For me, this would be "lusxvr/nanoALM"
# hf_model_name = "YOUR_HF_USER_NAME/nanoALM"

In [5]:
# nanoALM Imports (please check out the implementations in detail, that's where all the interessting stuff is!)
from data.collators import AlignmentCollator, AudioQACollator
from data.datasets import SAVEEDataset, AudioQADataset
from data.processors import get_audio_processor
from data.processors import get_tokenizer
from models.audio_language_model import AudioLanguageModel
import models.utils as utils

# Libraries
import math
import time
import torch

from tqdm import tqdm
import torch.optim as optim
import matplotlib.pyplot as plt
from dataclasses import dataclass
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
#Otherwise, the tokenizer will through a warning
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

torch.autograd.set_detect_anomaly(True)

torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

# To reload the modules if you change something in the code
%reload_ext autoreload
%autoreload 2

Using device: cuda


## Functions

### Get the dataloaders

In [6]:
def get_dataloaders(train_cfg, alm_cfg, tokenizer):
    # Create datasets
    audio_processor = get_audio_processor(alm_cfg)

    # text = "splitting datasets, disable in get_dataloaders function"
    # print(f"\n\033[38;5;05m{text}05m\033[0m")
    # Load and combine all training datasets
    combined_train_data = []
    for dataset_name in train_cfg.train_dataset_name:
        train_ds = load_dataset(
        path = train_cfg.train_dataset_path,
        name = dataset_name,
    )
        combined_train_data.append(train_ds['train'])
    train_ds = concatenate_datasets(combined_train_data)

    test_ds = load_dataset(train_cfg.test_dataset_path)
    train_ds = train_ds.shuffle(seed=0) # Shuffle the training dataset, so train and val get equal contributions from all concatinated datasets

    # Apply cutoff if specified
    if train_cfg.data_cutoff_idx is None:
        total_samples = len(train_ds)  # Use the entire dataset
    else:
        total_samples = min(len(train_ds), train_cfg.data_cutoff_idx)

    val_size = int(total_samples * train_cfg.val_ratio)
    train_size = total_samples - val_size

    train_dataset = AudioQADataset(train_ds.select(range(train_size)), tokenizer, audio_processor)
    val_dataset = AudioQADataset(train_ds.select(range(train_size, total_samples)), tokenizer, audio_processor)
    test_dataset = SAVEEDataset(test_ds, tokenizer, audio_processor)

    # Create collators
    alignment_collator = AlignmentCollator(tokenizer, alm_cfg.lm_max_length, audio_processor)
    aqa_collator = AudioQACollator(tokenizer)
    savee_collator = AudioQACollator(tokenizer)

    # Create dataloaders
    alignment_train_loader = DataLoader(
        train_dataset,
        batch_size=train_cfg.batch_size,
        shuffle=True,
        collate_fn=alignment_collator,
        num_workers=2,
        pin_memory=True,
        drop_last=True,
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=train_cfg.batch_size,
        shuffle=True,
        collate_fn=aqa_collator,
        num_workers=2,
        pin_memory=True,
        drop_last=True,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=train_cfg.batch_size,
        shuffle=False,
        collate_fn=aqa_collator,
        num_workers=2,
        pin_memory=True,
        drop_last=True,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=train_cfg.savee_batch_size,
        shuffle=False,
        collate_fn=savee_collator,
        pin_memory=True,
        )

    return alignment_train_loader, train_loader, val_loader, test_loader

### Prepare the testing function

In [7]:
def test_savee(model, tokenizer, test_loader, device):
    total_examples = 0
    correct_predictions = 0
    with torch.no_grad():
        for batch in test_loader:
            audio = batch['audio'].to(device)
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            correct_answer = tokenizer.batch_decode(labels, skip_special_tokens=True)

            gen = model.generate(input_ids, audio, attention_mask)
            model_output = tokenizer.batch_decode(gen, skip_special_tokens=True)

            is_correct = utils.check_multiple_choice_with_regex(model_output, correct_answer)

            total_examples += len(is_correct)
            if is_correct:
                correct_predictions += sum(is_correct)
    accuracy = correct_predictions / total_examples if total_examples > 0 else 0
    return accuracy

def get_avg_alignment(model, val_loader, device, epoch):
    """
    Validate the model's audio-text alignment on the validation set.
    This function computes the average alignment score over the validation set.
    It runs for a maximum of 20 batches to save time during training.
    """
    model.eval()
    total_alignment_score = 0

    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if i >= 20:  # 只驗證前20個batch以節省時間
                break
            audios = batch["audio"].to(device)
            input_ids = batch["input_ids"].to(device)
            alignment_score = model.validate_audio_text_alignment(input_ids, audios)
            total_alignment_score += alignment_score

    avg_alignment = total_alignment_score / min(20, len(val_loader))
    print(f"Epoch {epoch+1}: Average alignment score: {avg_alignment:.4f}")

    print(" ")
    model.train()
    return avg_alignment

### Prepare the training loop

#### Three-stage training (contrast training, generative training, instruction fine-tuning) 三段式訓練(對比訓練, 生成式訓練, 指令微調)

In [8]:
import torch.nn.functional as F
import torch.nn as nn
import torch.amp as GradScaler

from debug.debug_func import debug_contrastive_learning

# 改進對比學習訓練
def get_lr(it, max_lr, max_steps):
    min_lr = max_lr * 0.1
    warmup_steps = max_steps * 0.03
    # 1) linear warmup for warmup_iters steps
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    # 2) if it > lr_decay_iters, return min learning rate
    if it > max_steps:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff starts at 1 and goes to 0
    return min_lr + coeff * (max_lr - min_lr)

def get_contrastive_loss(audio_embeds, text_embeds, temperature=0.07):
    """
    標準、高效的對比學習損失 (CLIP Loss)。
    注意：輸入的 embeds 應該是池化後的 [B, D] 維度向量。
    """
    # --- 開始計算尺度對齊損失 ---

    # 1. 計算每個向量的 L2 範數 (沿著最後一個維度)
    #    detach() 是為了確保這個輔助損失的梯度只流向投影器，而不影響上游的編碼器或文字嵌入層的穩定性（可選但推薦）
    audio_norms = torch.norm(audio_embeds, p=2, dim=-1)
    text_norms = torch.norm(text_embeds, p=2, dim=-1)

    # 2. 計算批次內的平均範數
    mean_audio_norm = torch.mean(audio_norms)
    mean_text_norm = torch.mean(text_norms)

    # 3. 計算尺度對齊損失 (使用 MSE)
    #    目標是讓 mean_audio_norm 和 mean_text_norm 盡可能接近
    scale_loss = F.mse_loss(mean_audio_norm, mean_text_norm)

    # --- 結束計算尺度對齊損失 ---

    # --- 開始計算交叉熵損失 ---
    # 歸一化
    audio_embeds = F.normalize(audio_embeds, p=2, dim=-1)
    text_embeds = F.normalize(text_embeds, p=2, dim=-1)

    # 計算相似度矩陣
    # temperature 是一個重要的超參數，CLIP 論文中是可學習的，但固定值也可以
    logits_per_audio = torch.matmul(audio_embeds, text_embeds.T) / temperature
    logits_per_text = logits_per_audio.T

    # 創建標籤 (0, 1, 2, ..., B-1)
    labels = torch.arange(audio_embeds.shape[0]).to(logits_per_audio.device)

    # 對稱的交叉熵損失
    loss_a = F.cross_entropy(logits_per_audio, labels)
    loss_t = F.cross_entropy(logits_per_text, labels)

    contrastive_loss = (loss_a + loss_t) / 2

    # --- 結束計算交叉熵損失 ---


    # 4. 組合損失
    #    lambda_scale 是一個需要調整的超參數，用來平衡兩個損失的權重
    lambda_scale = 0.001  # 範例值，可以從 0.01, 0.1, 1.0 等開始嘗試
    total_loss = contrastive_loss + lambda_scale * scale_loss

    # 監控指標 (可選但推薦)
    with torch.no_grad():
        pos_sim = torch.diagonal(logits_per_audio * temperature).mean()
        mask = ~torch.eye(labels.shape[0], dtype=torch.bool, device=labels.device)
        neg_sim = (logits_per_audio * temperature)[mask].mean()

    return total_loss, contrastive_loss, scale_loss, {
        "loss": total_loss.item(),
        "pos_sim": pos_sim.item(), # 正樣本對的餘弦相似度
        "neg_sim": neg_sim.item()  # 負樣本對的餘弦相似度
    }

def train_step1_alignment(train_cfg, alm_cfg, model=None, tokenizer=None, device=None):
    # 凍結音頻編碼器和語言模型
    model.audio_encoder.audio_encoder.requires_grad_(False)
    model.decoder.requires_grad_(False)
    model.MP.requires_grad_(True)

    alignment_train_loader, _, val_loader, _ = get_dataloaders(train_cfg, alm_cfg, tokenizer)

    optimizer = optim.AdamW(model.MP.parameters(), lr=train_cfg.lr_mp, weight_decay=0.01)

    best_alignment = 0

    for epoch in range(train_cfg.stage1_epochs):
        model.train()
        total_contrastive_loss = 0  # 添加這個變數初始化
        total_scale_loss = 0  # 添加這個變數初始化

        for batch in tqdm(alignment_train_loader, desc=f"Stage1 Epoch {epoch+1}"):
            audios = batch["audio"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            optimizer.zero_grad()

            # 1. 音頻編碼 -> 投影
            with torch.no_grad():
                audio_features = model.audio_encoder.audio_encoder(audios, output_hidden_states=True)
            projected_audio_features = model.MP(audio_features.last_hidden_state)

            # 2. 文本編碼 - 修復這裡的問題
            with torch.no_grad():
                # 檢查 decoder 的 forward 方法簽名
                # 根據 language_model.py，應該傳入 x 而不是分別的參數
                text_embeds = model.decoder.token_embedding(input_ids)  # 直接獲取文本嵌入

                # 如果需要通過完整的 decoder，使用以下方式：
                # text_outputs, _ = model.decoder(text_embeds, attention_mask=attention_mask)
                # text_embeds = text_outputs  # 使用輸出的嵌入

            # 3. 池化操作 (Pooling)
            # 音頻池化
            audio_pooled = projected_audio_features.mean(dim=1)  # [B, D]

            # 文本池化 - 修復維度問題
            # text_embeds 現在是 [B, seq_len, hidden_dim]
            if attention_mask is not None:
                # 根據 attention_mask 來安全地做平均池化
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(text_embeds.size()).float()
                sum_embeddings = torch.sum(text_embeds * input_mask_expanded, 1)
                sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
                text_pooled = sum_embeddings / sum_mask  # [B, D]
            else:
                text_pooled = text_embeds.mean(dim=1)  # [B, D]

            # 如果維度仍然不匹配，添加投影層
            if audio_pooled.shape[-1] != text_pooled.shape[-1]:
                # 創建一個投影層來匹配維度
                if not hasattr(model, 'text_projection'):
                    model.text_projection = nn.Linear(text_pooled.shape[-1], audio_pooled.shape[-1]).to(device)
                text_pooled = model.text_projection(text_pooled)

            # 4. 計算對比損失
            loss, contrastive_loss, scale_loss, metrics = get_contrastive_loss(audio_pooled, text_pooled)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.MP.parameters(), max_norm=1.0)
            optimizer.step()

            total_contrastive_loss += contrastive_loss.item()
            total_scale_loss += scale_loss.item()

        avg_contrastive_loss = total_contrastive_loss / len(alignment_train_loader)
        avg_scale_loss = total_scale_loss / len(alignment_train_loader)
        print(f"\nEpoch {epoch+1}: Total Loss {loss:.4f}, Contrastive Loss {avg_contrastive_loss:.4f}, Scale Loss {avg_scale_loss:.4f}")

        avg_alignment = get_avg_alignment(model, val_loader, device, epoch)

        if avg_alignment > best_alignment:
            best_alignment = avg_alignment
            model.save_pretrained(save_directory=f"{alm_cfg.alm_checkpoint_path}/stage1_best")
            print(f"  New best alignment: {best_alignment:.4f}")

    print(f"Stage 1 completed! Best alignment: {best_alignment:.4f}")
    return model

def train_step2_pretraining(train_cfg, alm_cfg, stage1_model=None, tokenizer=None, device=None):
    print("=== Stage 2: Language Model Pretraining ===")

    # 使用傳入的 tokenizer，不要重建
    _, train_loader, val_loader, test_loader = get_dataloaders(train_cfg, alm_cfg, tokenizer)

    model = stage1_model
    # 冻结/解冻
    model.audio_encoder.audio_encoder.requires_grad_(False)
    model.MP.requires_grad_(True)
    model.decoder.requires_grad_(True)  # 或僅解凍頂層幾層

    # 調小 decoder LR
    param_groups = [
        {'params': [p for p in model.MP.parameters() if p.requires_grad], 'lr': train_cfg.lr_mp * 0.05},
        {'params': [p for p in model.decoder.parameters() if p.requires_grad], 'lr': train_cfg.lr_backbones},
    ]
    optimizer = optim.AdamW(param_groups, weight_decay=0.01)
    scaler = torch.amp.GradScaler(device=device)

    batch_losses = []
    best_loss = float('inf')
    global_step = 0
    total_steps = len(train_loader) * train_cfg.stage2_epochs

    for epoch in range(train_cfg.stage2_epochs):
        model.train()
        total_train_loss = 0.0

        for batch in tqdm(train_loader, desc=f"Stage2 Epoch {epoch+1}"):
            audios = batch["audio"].to(device)
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            optimizer.zero_grad(set_to_none=True)

            if alm_cfg.dtype == torch.float16:
                with torch.autocast(device_type='cuda', dtype=torch.float16):
                    _, loss = model(input_ids, audios, attention_mask=attention_mask, labels=labels)
            else:
                _, loss = model(input_ids, audios, attention_mask=attention_mask, labels=labels)

            scaler.scale(loss).backward()

            # 調整 LR
            optimizer.param_groups[0]['lr'] = get_lr(global_step, param_groups[0]['lr'], total_steps)
            optimizer.param_groups[1]['lr'] = get_lr(global_step, param_groups[1]['lr'], total_steps)

            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(
                [p for p in model.parameters() if p.requires_grad and p.grad is not None],
                max_norm=1.0
            )
            scaler.step(optimizer)
            scaler.update()

            batch_loss = loss.item()
            total_train_loss += batch_loss
            batch_losses.append(batch_loss)
            global_step += 1

        avg_train_loss = total_train_loss / len(train_loader)
        if avg_train_loss < best_loss:
            best_loss = avg_train_loss
            model.save_pretrained(save_directory=f"{alm_cfg.alm_checkpoint_path}/stage2_best")

        avg_alignment = get_avg_alignment(model, val_loader, device, epoch)
        print(f"\nEpoch {epoch+1}/{train_cfg.stage2_epochs} | Loss: {avg_train_loss:.4f} | Alignment: {avg_alignment:.4f}")

    model.save_pretrained(save_directory=f"{alm_cfg.alm_checkpoint_path}/stage2_final")
    print("Stage 2 completed!")
    plt.plot(batch_losses, label='Train Loss')
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.title('Loss Curve')
    plt.grid(True)
    plt.legend()
    plt.show()

    return model

def train_step3_instruction_tuning(train_cfg, alm_cfg, stage2_model=None, tokenizer=None, device=None):
    """第三步：指令微调"""
    print("=== Stage 3: Instruction Tuning ===")

    _, train_loader, val_loader, test_loader = get_dataloaders(train_cfg, alm_cfg, tokenizer)
    scaler = torch.amp.GradScaler(device=device)

    model = stage2_model
    # 全部解冻，使用较小学习率
    for param in model.parameters():
        param.requires_grad = True

    print(f"Stage 3: Training all {sum(p.numel() for p in model.parameters()):,} parameters")

    # 更小的学习率
    param_groups = [
        {'params': model.MP.parameters(), 'lr': train_cfg.lr_mp * 0.01},
        {'params': model.decoder.parameters(), 'lr': train_cfg.lr_backbones * 0.1},
        {'params': model.audio_encoder.audio_encoder.parameters(), 'lr': train_cfg.lr_backbones * 0.01}
    ]
    optimizer = optim.AdamW(param_groups)

    if train_cfg.compile:
        model = torch.compile(model)

    # 这里可以使用原来的训练循环，但数据应该是指令格式
    # 暂时使用相同的数据格式
    best_accuracy = 0
    global_step = 0

    for epoch in range(train_cfg.stage3_epochs):
        model.train()
        total_train_loss = 0

        for batch in tqdm(train_loader, desc=f"Stage3 Epoch {epoch+1}"):
            audios = batch["audio"].to(device)
            input_ids = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            optimizer.zero_grad()

            if alm_cfg.dtype == torch.float16:
                with torch.autocast(device_type='cuda', dtype=torch.float16):
                    _, loss = model(input_ids, audios, attention_mask=attention_mask, labels=labels)
            else:
                _, loss = model(input_ids, audios, attention_mask=attention_mask, labels=labels)


            scaler.unscale_(optimizer)
            # 2. 對 unscale 後的梯度進行裁剪 (max_norm=1.0 是一個常用的值)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            # --- 修改結束 ---

            # 3. Scaler 執行優化器步驟 (如果梯度沒有 inf/nan)
            scaler.step(optimizer)
            # 4. 更新 scaler 的縮放因子
            scaler.update()

            batch_loss = loss.item()
            total_train_loss += batch_loss

            if global_step % 50 == 0:
                print(f"Stage3 Step: {global_step}, Instruction Loss: {batch_loss:.4f}")

            global_step += 1

        avg_train_loss = total_train_loss / len(train_loader)

        # 评估性能
        if train_cfg.eval_in_epochs:
            accuracy = test_savee(model, tokenizer, test_loader, device)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                model.save_pretrained(save_directory=f"{alm_cfg.alm_checkpoint_path}/stage3_best")
            print(f"Stage3 Epoch {epoch+1}/{train_cfg.stage3_epochs} | Loss: {avg_train_loss:.4f} | Accuracy: {accuracy:.4f}")
        else:
            print(f"Stage3 Epoch {epoch+1}/{train_cfg.stage3_epochs} | Instruction Loss: {avg_train_loss:.4f}")

    # 保存最终模型
    model.save_pretrained(save_directory=f"{alm_cfg.alm_checkpoint_path}/final_model")
    print("Stage 3 completed!")
    return model

def train_three_stages(train_cfg, alm_cfg, device=None):
    """完整的三阶段训练"""
    print("Starting Three-Stage Training Pipeline")

    # 第一阶段：模态投影器对齐
    stage1_model = train_step1_alignment(train_cfg, alm_cfg, device=device)

    # 第二阶段：语言模型预训练
    stage2_model = train_step2_pretraining(train_cfg, alm_cfg, stage1_model, device=device)

    # 第三阶段：指令微调
    final_model = train_step3_instruction_tuning(train_cfg, alm_cfg, stage2_model, device=device)

    print("=== Training Pipeline Completed! ===")
    return stage1_model, stage2_model, final_model


# # 替换原来的训练调用
# alm_cfg = ALMConfig()
# train_cfg = TrainConfig()

# # 运行三阶段训练
# final_model = train_three_stages(train_cfg, alm_cfg)

## Debug

In [9]:
# !python ./debug/debug_forward.py

## Lets run the training!

In [10]:
import os
from models.config import ALMConfig, TrainConfig

# 要創建的目錄路徑
dir_name = ALMConfig.alm_checkpoint_path

try:
    os.mkdir(dir_name)
    print(f"Directory '{dir_name}' created successfully.")
except FileExistsError:
    print(f"Directory '{dir_name}' already exists.")
except FileNotFoundError:
    print(f"Parent directory does not exist for '{dir_name}'.")
except Exception as e:
    print(f"An error occurred: {e}")

alm_cfg = ALMConfig()
train_cfg = TrainConfig()
dtype = alm_cfg.dtype
device = alm_cfg.device
print(f"Using device: {device}")


tokenizer = get_tokenizer(alm_cfg.lm_tokenizer)

# 創建一個帶有新 token 的模型實例
model = None
stage1_model = None

if train_cfg.resume_from_alm_checkpoint:
    checkpoint_path = "../stage1"
    # checkpoint_path = "./checkpoints/stage1_best"
    stage1_model = AudioLanguageModel.from_pretrained(checkpoint_path, tokenizer=tokenizer)

else:
    model = AudioLanguageModel(alm_cfg, load_from_HF=True, tokenizer=tokenizer)


Directory 'checkpoints' already exists.
Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


load model from local
Initializing empty audio encoder: openai/whisper-small.en


In [11]:
if model != None:
    stage1_model = train_step1_alignment(train_cfg, alm_cfg, model, tokenizer, device)
    stage1_model.save_pretrained("/content/")

In [12]:
stage2_model = train_step2_pretraining(train_cfg, alm_cfg, stage1_model, tokenizer, device)
stage2_model.save_pretrained("/content/")

=== Stage 2: Language Model Pretraining ===
AudioProcessor_from_HF initialized with model: <class 'transformers.models.whisper.processing_whisper.WhisperProcessor'>
  Target feature frames from cfg: 1500
  Using model sampling rate: 16000, hop_length: 160, n_fft: 400
  Calculated max raw audio samples for processor: 240240


Stage2 Epoch 1:   0%|          | 0/205 [00:00<?, ?it/s]

Debug(AudioLanguageModel): text_embeds:  torch.Size([4, 54, 2048])
Debug(AudioLanguageModel): audio_embeds:  torch.Size([4, 25, 2048])
Debug(AudioLanguageModel): Input Embeds = 
: shape=torch.Size([4, 78, 2048]), dtype=torch.float32, min=-4.2626, max=4.2608, mean=-0.0002
Debug(AudioLanguageModel): Decoder Output Embeds = 
: shape=torch.Size([4, 78, 2048]), dtype=torch.float32, min=-54.5700, max=48.1333, mean=0.0052
Debug(AudioLanguageModel): input_ids:  torch.Size([4, 54]) input_ids:  tensor([[    1,  4093,   198,  1780,   314,  1137,   281,   451,  8389,    47,
           216, 49152,     2,   198,     1,   520,  9531,   198, 34763,    28,
          1303,   549,   820,  1272,    30, 16912,    28,  6820,   617,   260,
         12541,   314,    47,   657,   506,   549,    30,     0,     2,   198,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [    1,  4093,   198,  1780,   314,  1137,   281,   451,  8389,    47

Stage2 Epoch 1:   0%|          | 1/205 [00:03<11:48,  3.47s/it]

Debug(AudioLanguageModel): text_embeds:  torch.Size([4, 69, 2048])
Debug(AudioLanguageModel): audio_embeds:  torch.Size([4, 25, 2048])
Debug(AudioLanguageModel): Input Embeds = 
: shape=torch.Size([4, 93, 2048]), dtype=torch.float32, min=-4.7663, max=4.9278, mean=-0.0003
Debug(AudioLanguageModel): Decoder Output Embeds = 
: shape=torch.Size([4, 93, 2048]), dtype=torch.float32, min=-54.5700, max=48.7536, mean=0.0036
Debug(AudioLanguageModel): input_ids:  torch.Size([4, 69]) input_ids:  tensor([[    1,  4093,   198,  1780,   314,  1137,   281,   451,  8389,    47,
           216, 49152,     2,   198,     1,   520,  9531,   198,  6910,    28,
           346,   699,   787,  1542,  2166, 15411,   502,   359,  1035,  1165,
         13645,    30,     0,     2,   198,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,

...

Stage2 Epoch 1:  55%|█████▍    | 112/205 [02:27<02:02,  1.32s/it]

Debug(AudioLanguageModel): text_embeds:  torch.Size([4, 33, 2048])
Debug(AudioLanguageModel): audio_embeds:  torch.Size([4, 25, 2048])
Debug(AudioLanguageModel): Input Embeds = 
: shape=torch.Size([4, 57, 2048]), dtype=torch.float32, min=-4.5374, max=4.3747, mean=-0.0002
Debug(AudioLanguageModel): Decoder Output Embeds = 
: shape=torch.Size([4, 57, 2048]), dtype=torch.float32, min=-54.5700, max=50.2288, mean=0.0063
Debug(AudioLanguageModel): input_ids:  torch.Size([4, 33]) input_ids:  tensor([[    1,  4093,   198,  1780,   314,  1137,   281,   451,  8389,    47,
           216, 49152,     2,   198,     1,   520,  9531,   198,  1206,  1303,
          1272, 21111,   346,    30,   216,  1206,  1137,   346,  1250,    30,
             0,     2,   198],
        [    1,  4093,   198,  1780,   314,  1137,   281,   451,  8389,    47,
           216, 49152,     2,   198,     1,   520,  9531,   198,   657,  2761,
           982,  2395,   288,   746,   715, 13278,   351,   346,    30,     0,
     

Stage2 Epoch 1:  55%|█████▌    | 113/205 [02:29<02:01,  1.32s/it]

Debug(AudioLanguageModel): text_embeds:  torch.Size([4, 97, 2048])
Debug(AudioLanguageModel): audio_embeds:  torch.Size([4, 25, 2048])


KeyboardInterrupt: 

In [ ]:
final_model = train_step3_instruction_tuning(train_cfg, alm_cfg, stage2_model, tokenizer, device)
final_model.save_pretrained("/content/")

As you can see the model trains, so feel free to play around with the architecture or data! Let us know what you build with it!

PS: If you want to test the model, check out generate.py to see how to do inference with it

## Test

In [ ]:
!cp /content/drive/MyDrive/nanoALM/output_txt1.wav /content/output_txt1.wav
!cp /content/drive/MyDrive/nanoALM/output_txt2.wav /content/output_txt2.wav
!cp /content/drive/MyDrive/nanoALM/output_txt3.wav /content/output_txt3.wav

In [ ]:
!cp ../model.safetensors /content/drive/MyDrive/nanoALM/model.safetensors
!cp ../config.json /content/drive/MyDrive/nanoALM/config.json
!cp ./model.safetensors /content/drive/MyDrive/nanoALM
!cp ./config.json /content/drive/MyDrive/nanoALM

In [ ]:
# final_model.save_pretrained("/content/")
!python generate.py --checkpoint ../ --audio ../output_txt1.wav

In [ ]:
!python generate.py --checkpoint ../ --audio ../output_txt2.wav

In [ ]:
!python generate.py --checkpoint ../ --audio ../output_txt3.wav

In [ ]:
!python generate.py --checkpoint ../ --audio ../output_txt3.wav

In [ ]:
import librosa

messages = [
    {"role": "user", "content": "What is said in this audio? <AUDIO>"},
    {"role": "assistant", "content": ""},  # 讓模板能加上生成提示
]
full_input_ids = tokenizer.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True
)
full_input_ids = torch.tensor(full_input_ids, dtype=torch.long).to(device)
if full_input_ids.dim() == 1:
    full_input_ids = full_input_ids.unsqueeze(0)

generations = 5
max_new_tokens = 100

try:
    audio_array, sr = librosa.load("../output_txt1.wav", sr=16000)
    # 轉換為 torch tensor 並添加 batch 維度
    audio_tensor = torch.tensor(audio_array, dtype=torch.float32)
    if audio_tensor.dim() == 1:
        audio_tensor = audio_tensor.unsqueeze(0)  # 添加 channel 維度

    # 使用 audio_processor 處理音頻
    ap = get_audio_processor(alm_cfg)
    audio_t = ap(audio_array, sr).unsqueeze(0).to(device)
except Exception as e:
    print(f"Error loading audio file: {e}")
    print("Please check if the audio file exists and is in a supported format.")

tested_model = final_model if 'final_model' in globals() and final_model is not None else stage2_model
tested_model.to(device).eval()

print("\nInput:\n ", messages[0]["content"], "\n\nOutputs:")
for i in range(generations):
    gen = tested_model.generate(
        full_input_ids,
        audio_t,
        max_new_tokens=max_new_tokens,
        greedy=True,
        top_k=10,
        top_p=0.8,
        temperature=0.3
    )
    out = tokenizer.batch_decode(gen, skip_special_tokens=True)[0]
    print(f"  >> Generation {i+1}: {out}")